# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Q1
limit = 5
generator = square_root_generator(limit)
sum_of_outputs = sum(sqrt_value for sqrt_value in generator)
print("Sum of outputs for limit = 5:", sum_of_outputs)

# Q2
generator = square_root_generator(13)
thirteenth_value = None
for i, sqrt_value in enumerate(generator, 1):
    if i == 13:
        thirteenth_value = sqrt_value
        break
print("13th number yielded by the generator:", thirteenth_value)


Sum of outputs for limit = 5: 8.382332347441762
13th number yielded by the generator: 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [ ]:
import duckdb

# Create connection to DuckDB
con = duckdb.connect(database=':memory:', read_only=False)

# Define the first generator
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

# Create a table for the first generator data
con.execute("""
CREATE TABLE people (
    ID INTEGER,
    Name VARCHAR,
    Age INTEGER,
    City VARCHAR,
    Occupation VARCHAR
)
""")

# Insert data from the first generator into the table
for person in people_1():
    con.execute("INSERT INTO people VALUES (?, ?, ?, ?, NULL)", (person["ID"], person["Name"], person["Age"], person["City"]))

# Define the second generator
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# Append data from the second generator to the same table
for person in people_2():
    con.execute("INSERT INTO people VALUES (?, ?, ?, ?, ?)", (person["ID"], person["Name"], person["Age"], person["City"], person["Occupation"]))

# Calculate the sum of ages of all people
result_1 = con.execute("SELECT SUM(Age) FROM people")
sum_of_ages_1 = result_1.fetchone()[0]
print("Sum of ages of all people after loading the first generator:", sum_of_ages_1)

# Calculate the sum of ages of all people after appending the second generator
result_2 = con.execute("SELECT SUM(Age) FROM people")
sum_of_ages_2 = result_2.fetchone()[0]
print("Sum of ages of all people after appending the second generator:", sum_of_ages_2)

# Close the connection
con.close()


Sum of ages of all people after loading the first generator: 353
Sum of ages of all people after appending the second generator: 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)
import duckdb

# Create connection to DuckDB
con = duckdb.connect(database=':memory:', read_only=False)

# Define the first generator
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

# Create a table for the data with primary key constraint
con.execute("""
CREATE TABLE people (
    ID INTEGER PRIMARY KEY,
    Name VARCHAR,
    Age INTEGER,
    City VARCHAR,
    Occupation VARCHAR
)
""")

# Insert data from the first generator into the table
for person in people_1():
    con.execute("INSERT INTO people VALUES (?, ?, ?, ?, NULL)", (person["ID"], person["Name"], person["Age"], person["City"]))

# Define the second generator
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# Merge data from the second generator into the table
for person in people_2():
    con.execute("INSERT OR REPLACE INTO people VALUES (?, ?, ?, ?, ?)", (person["ID"], person["Name"], person["Age"], person["City"], person["Occupation"]))

# Calculate the sum of ages of all people
result = con.execute("SELECT SUM(Age) FROM people")
sum_of_ages = result.fetchone()[0]
print("Sum of ages of all people loaded:", sum_of_ages)

# Close the connection
con.close()


Sum of ages of all people loaded: 266


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX